
## Work in progress - openCoesione with Comune as key 

In [3]:
## Runs with "daf" enviroment in repo

%config Completer.use_jedi = False #fix TAB slowness with big frames

import numpy as np
import pandas as pd
import seaborn as sns
import glob
import gc #garbage collector
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (20,12)

def load_and_stack_data(prefix):
    fileList = glob.glob('./../pac_opencoesione/%s*.csv' % prefix)
    
    if prefix == 'progetti':
        dtypesDict = pd.read_csv(
            './openCoesione_dtypes.csv', index_col='Variabile', squeeze=True).to_dict()
        dateCols = [name for name in dtypesDict.keys() if dtypesDict[name] == 'datetime64']
        for name in dateCols: del dtypesDict[name]
        frames = list(map(lambda f: pd.read_csv(
            f, sep=';', dtype=dtypesDict, index_col=0, na_values=[' ', '  ']), fileList))
        data = pd.concat(frames, axis=0)
        data[dateCols] = data[dateCols].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
        
        # drop duplicates including index when checking
        indexColName = data.index.name
        data.reset_index(inplace=True)
        data.drop_duplicates(inplace=True)
        data.set_index(indexColName, inplace=True)
        
        # clean remaining duplicate indexes by picking the row that has fewer N/A
        bDuplicate = data.index.duplicated(keep=False)
        if any(bDuplicate):
            print('%i duplicate indexes found' % sum(bDuplicate))
            duplRows = data[bDuplicate].copy()
            duplRows['NumberNA'] = duplRows.isna().sum(axis=1)
            duplRows.sort_values('NumberNA', inplace=True)
            # recomibine into final frame
            data = pd.concat([data[~bDuplicate], duplRows[duplRows.index.duplicated(keep='first')]])
        
    else:
        # infer types for other datasets (temporary)
        frames = list(map(lambda f: pd.read_csv(f, sep=';', index_col=False), fileList))
        data = pd.concat(frames, axis=0)
        
        # drop duplicates excluding index
        data.drop_duplicates(inplace=True)
        
    print('Imported %s' % prefix)
    return data



In [4]:
# load soggetti
listaMise =  pd.read_csv('./../MISE Aiuti/data/D_general/ListIvaCodFisc.csv', sep=';', index_col=False)
soggetti = load_and_stack_data('soggetti')


/home/davide/anaconda3/envs/daf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (4,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Imported soggetti


In [29]:
print(len(listaMise.values[listaMise.isin(soggetti.OC_CODICE_FISCALE_SOGG.values)]))


13651


## join: pagamenti + (progetti + soggetti==2) e luoghi

In [ ]:
# Step 1: JOIN soggetti-progetti // soggetti might have more than 1 acting entity, 
# initally drop them  %TODO: allow other progr soggetti and split payements

soggettiAttuatori = soggetti[
    (soggetti.SOGG_PROGR_RUOLO == 1) & (soggetti.SOGG_COD_RUOLO == 2)].set_index('COD_LOCALE_PROGETTO')  
assert not any(soggettiAttuatori.index.duplicated()), 'Attuatori duplicati'

newUnion = progetti[progetti.OC_FLAG_REGIONE_UNICA=='1'].join(soggettiAttuatori)
print('---CONSIDERATI PROGETTI con solo una regione')
firstJoinTotPagamenti = newUnion.TOT_PAGAMENTI.sum()

del progetti
gc.collect()

In [ ]:
# Step 2: JOIN progetti-luoghi // luoghi might have more than one row per project,
# initally drop them  %TODO: allow other luoghi, check match with soggetti and split payements

luoghiMatch = luoghi[luoghi.OC_FLAG_CAP_PROG==1].set_index('COD_LOCALE_PROGETTO')

luoghiMatch = luoghiMatch[~luoghiMatch.index.duplicated(keep='first')] #TODO: analyse repeated luoghi 
#assert not any(luoghiMatch.index.duplicated()), 'Luoghi duplicati'

newUnion = newUnion.join(luoghiMatch)
assert newUnion.TOT_PAGAMENTI.sum() == firstJoinTotPagamenti, 'Inconsistent join'

In [ ]:
# Step 3: JOIN pagamenti-progetti // fill pagamenti data with the above info

# check pagamenti sums
pagamenti = load_and_stack_data('pagamenti')
origSum = pagamenti['TOT_PAGAMENTI'].sum() # ok, around 60B

In [ ]:
#newUnion.join(pagamenti.set_index('COD_LOCALE_PROGETTO'),  lsuffix='all_')['TOT_PAGAMENTI'].sum()
newUnion = newUnion.join(pagamenti.set_index('COD_LOCALE_PROGETTO'),  lsuffix='_summary')

del pagamenti
gc.collect()
#print(pagamenti['TOT_PAGAMENTI'].sum())
#print(newUnion['TOT_PAGAMENTI'].sum())


#newUnion.head().join(pagamenti.set_index('COD_LOCALE_PROGETTO'),  lsuffix='all_').columns #['TOT_PAGAMENTI_all']

In [ ]:
multiSoggetto = soggetti[(soggetti.SOGG_PROGR_RUOLO > 1) & (soggetti.SOGG_COD_RUOLO == 2)] 

bProgettoMultiSogg = 

print(len(multiSoggettoKeys))


#print(soggetti.COD_DIMENSIONE_SOGG.unique())
#print(soggetti.groupby(['COD_DIMENSIONE_SOGG', 'DESCR_FORMA_GIURIDICA_SOGG']).count())
#print(soggetti[soggetti.COD_DIMENSIONE_SOGG.isnull()].groupby('DESCR_FORMA_GIURIDICA_SOGG')['OC_CODICE_FISCALE_SOGG'].count())

In [ ]:
# Summary on beneficiary types
soggetti.groupby('DESCR_FORMA_GIURIDICA_SOGG')['OC_CODICE_FISCALE_SOGG'].nunique().sort_values(ascending=False)